# 任务3：TFIDF提取与分类
TFIDF（词频-逆文档频率）是一种常见的文本表示方法，可以用于文本分类任务。TFIDF将文本表示为词项的权重向量，其中每个词项的权重由其在文本中出现的频率和在整个语料库中出现的频率共同决定。TFIDF可以反映出词项在文本中的重要程度，越是常见的词项权重越低，越是稀有的词项权重越高。

- 步骤1：使用sklearn中的TfidfVectorizer类提取训练集和测试集的特征，
- 步骤2：使用KNN/LR/SVM等分类器对训练集进行训练，并对验证集和测试集进行预测，评估模型的性能。
- 步骤3：通过上述步骤，请回答下面问题
    - max_df: 用于过滤掉高频词项，在[0.0, 1.0]之间表示比例；
     -min_df: 用于过滤掉低频词项，在[0.0, 1.0]之间表示比例；
    - max_features: 用于限制提取特征的数量，默认为None。
    - ngram_range: 用于指定提取n元语法特征时n值范围，默认为(1, 1)，即只提取单个词项。
    - stop_words: 用于指定停用词列表，默认为None。
    - norm: 用于指定归一化方法，默认为’l2’范数。
    - use_idf: 是否使用逆文档频率计算权重，默认为True。
    - smooth_idf: 是否平滑逆文档频率计算，默认为True
    - TFIDF中可以设置哪些参数，如何影响到提取的特征？TfidfVectorizer类中可以设置以下参数：
    - KNN/LR/SVM的精度对比：根据实验结果，比较三种分类器在验证集和测试集上预测正确率、召回率、F1值等指标，并分析各自优缺点。

In [2]:
import pandas as pd
import jieba
import matplotlib.pyplot as plt

# 读取数据集，这里是直接联网读取，也可以通过下载文件，再读取
data_dir = 'https://mirror.coggle.club/dataset/coggle-competition/'
train_data = pd.read_csv(data_dir + 'intent-classify/train.csv', sep='\t', header=None)
test_data = pd.read_csv(data_dir + 'intent-classify/test.csv', sep='\t', header=None)

cn_stopwords = pd.read_csv('https://mirror.coggle.club/stopwords/baidu_stopwords.txt', header=None)[0].values

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer 
import jieba  

tfidf = TfidfVectorizer(tokenizer=jieba.lcut,stop_words=list(cn_stopwords) ) 

train_tfidf = tfidf.fit_transform(train_data[0])  
test_tfidf = tfidf.transform(test_data[0])   

/Users/samprasyuan/miniforge3/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/samprasyuan/miniforge3/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'", 'a', 'ain', 'aren', 'c', 'couldn', 'd', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'i', 'isn', 'll', 'm', 'mon', 's', 'shouldn', 't', 've', 'wasn', 'weren', 'won', 'wouldn', '下', '不', '为什', '什', '今', '使', '先', '却', '只', '唷', '啪', '喔', '天', '好', '後', '最', '漫', '然', '特', '特别', '见', '设', '说', '达', '面', '麽', '－'] not in stop_words.
  warnings.warn(


In [5]:
test_data

,0
0,回放CCTV2的消费主张
1,给我打开玩具房的灯
2,循环播放赵本山的小品相亲来听
3,15号上午10点带孩子去海洋馆的行程帮我制定下。
4,把智能扫地机器人关掉
...,...
2995,是否能找一首2019年的抖音歌曲播放下呢
2996,下午三点有哪个台放大话西游吗帮我看下
2997,随机播放一首古筝弹奏的曲子可以吗
2998,美食纪录片螃蟹的征途给我找一下


In [7]:
from sklearn.linear_model import LogisticRegression  
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.svm import LinearSVC   
from sklearn.metrics import classification_report 
from sklearn.model_selection import cross_val_predict  

cv_pred  = cross_val_predict(LogisticRegression(), train_tfidf, train_data[1])  
print(classification_report(train_data[1], cv_pred)  )

                       precision    recall  f1-score   support

         Alarm-Update       0.98      0.93      0.96      1264
           Audio-Play       0.74      0.50      0.60       226
       Calendar-Query       0.99      0.95      0.97      1214
        FilmTele-Play       0.70      0.93      0.80      1355
HomeAppliance-Control       0.94      0.97      0.96      1215
           Music-Play       0.88      0.87      0.87      1304
                Other       0.39      0.07      0.11       214
         Radio-Listen       0.94      0.89      0.91      1285
       TVProgram-Play       0.72      0.45      0.55       240
         Travel-Query       0.92      0.96      0.94      1220
           Video-Play       0.90      0.87      0.89      1334
        Weather-Query       0.92      0.96      0.94      1229

             accuracy                           0.89     12100
            macro avg       0.84      0.78      0.79     12100
         weighted avg       0.89      0.89      0.89 

In [8]:
cv_pred = cross_val_predict(
    LinearSVC(),
    train_tfidf, train_data[1]
)
print(classification_report(train_data[1], cv_pred))

/Users/samprasyuan/miniforge3/lib/python3.10/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/samprasyuan/miniforge3/lib/python3.10/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/samprasyuan/miniforge3/lib/python3.10/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/samprasyuan/miniforge3/lib/python3.10/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.wa

                       precision    recall  f1-score   support

         Alarm-Update       0.97      0.95      0.96      1264
           Audio-Play       0.64      0.71      0.67       226
       Calendar-Query       0.98      0.97      0.98      1214
        FilmTele-Play       0.81      0.89      0.85      1355
HomeAppliance-Control       0.97      0.98      0.98      1215
           Music-Play       0.90      0.89      0.89      1304
                Other       0.31      0.25      0.27       214
         Radio-Listen       0.94      0.90      0.92      1285
       TVProgram-Play       0.66      0.62      0.64       240
         Travel-Query       0.95      0.98      0.97      1220
           Video-Play       0.92      0.88      0.90      1334
        Weather-Query       0.96      0.97      0.97      1229

             accuracy                           0.91     12100
            macro avg       0.83      0.83      0.83     12100
         weighted avg       0.91      0.91      0.91 